In [1]:
import time
import datetime
import pandas as pd
from pandas import DataFrame
import numpy as np
import scipy
from numpy import array, zeros
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, make_scorer, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from math import exp
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
Xcsv = pd.read_csv('out_featers.csv', index_col='match_id')

In [3]:
n_samples = Xcsv.shape[0]#10000#
X = Xcsv[0:n_samples]
index = X.index
X = X.fillna(0)#, inplace=True)
Y = X['radiant_win']
Z = Y[0:n_samples]
X.drop(['radiant_win',
        'duration',
        'tower_status_radiant', 
        'tower_status_dire', 
        'barracks_status_dire', 
        'barracks_status_radiant', 
        'start_time'],
       axis=1, inplace=True)

In [4]:
heroes = [name for name in Xcsv.columns.values if '_hero' in name]
Xh = X[heroes]
Xl = X['lobby_type']
max_hero = Xh.values.max()
g_params = [column.replace('r1_', '') for column in Xcsv.columns if 'r1' in column]

In [5]:
X.drop(heroes, axis=1, inplace=True)
X.drop('lobby_type', axis=1, inplace=True)
Xh_team = pd.get_dummies(Xh, columns=heroes)
Xl_dummy = pd.get_dummies(Xl).values

In [6]:
X_radiant = DataFrame(index=index)
X_dire = DataFrame(index=index)
for person in range(1, max_hero + 1):
    dire = zeros((n_samples), dtype=int) 
    radiant =  zeros((n_samples), dtype=int)
    for num_team in range(1, 6):
#        d_name = f'd{num_team}_hero_{gamer}'
#        r_name = f'r{num_team}_hero_{gamer}'
        try:
            dire += Xh_team[f'd{num_team}_hero_{person}'].values
        except:
            pass
        try:
            radiant += Xh_team[f'r{num_team}_hero_{person}'].values
        except:
            pass
    try:
        X_radiant[f'{person}'] = radiant
    except:
        pass
    try:
        X_dire[f'{person}'] = dire
    except:
        pass
X_gamers = X_radiant - X_dire

In [7]:
scaler = MinMaxScaler()#StandardScaler()#
#XX = np.concatenate([X_gamers, Xl_dummy, X], axis=1)
XX = np.concatenate([X_radiant, X_dire, Xl_dummy, X], axis=1)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(
    XX, Y, test_size = 0.3,
    random_state = 241 )
X_train_scaled = scaler.fit_transform(X_train);
X_test_scaled = scaler.transform(X_test)

In [9]:
#logreg = LogisticRegression(n_jobs=-1, solver='sag')
logreg = LogisticRegression(C=0.25)
%time logreg.fit(X_train_scaled, Y_train);
s = logreg.score(X_test_scaled, Y_test)
print(s)

CPU times: user 6.32 s, sys: 10 ms, total: 6.33 s
Wall time: 6.75 s
0.6833624738592341


In [10]:
cv = KFold(n_splits=5, shuffle=True, random_state=241)
scorer = make_scorer(roc_auc_score)
params = {'C' : [(i/20) for i in range(3,8)]}
grid = GridSearchCV(LogisticRegression(random_state=241), params, cv=cv, n_jobs=-1, scoring=scorer, verbose=False)
%time grid.fit(X_train_scaled, Y_train)

CPU times: user 12.3 s, sys: 220 ms, total: 12.6 s
Wall time: 60 s


GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.15, 0.2, 0.25, 0.3, 0.35]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(roc_auc_score), verbose=False)

In [11]:
grid.best_params_

{'C': 0.15}

In [12]:
grid.best_score_

0.6799123974174485

In [13]:
type(X_gamers)

pandas.core.frame.DataFrame

In [14]:
[column.replace('r1_', '') for column in Xcsv.columns if 'r1' in column]

['hero',
 'level',
 'xp',
 'gold',
 'lh',
 'kills',
 'deaths',
 'items',
 'first_blood_player1']

In [15]:
matrixes = {column.replace('r1_', ''):DataFrame(index=index)
            for column in Xcsv.columns if 'r1' in column}

In [16]:
matrixes.keys()

dict_keys(['hero', 'level', 'xp', 'gold', 'lh', 'kills', 'deaths', 'items', 'first_blood_player1'])

In [17]:
columns = [col for col in Xcsv.columns]

In [18]:
for col in columns:
    print(col)

start_time
lobby_type
r1_hero
r1_level
r1_xp
r1_gold
r1_lh
r1_kills
r1_deaths
r1_items
r2_hero
r2_level
r2_xp
r2_gold
r2_lh
r2_kills
r2_deaths
r2_items
r3_hero
r3_level
r3_xp
r3_gold
r3_lh
r3_kills
r3_deaths
r3_items
r4_hero
r4_level
r4_xp
r4_gold
r4_lh
r4_kills
r4_deaths
r4_items
r5_hero
r5_level
r5_xp
r5_gold
r5_lh
r5_kills
r5_deaths
r5_items
d1_hero
d1_level
d1_xp
d1_gold
d1_lh
d1_kills
d1_deaths
d1_items
d2_hero
d2_level
d2_xp
d2_gold
d2_lh
d2_kills
d2_deaths
d2_items
d3_hero
d3_level
d3_xp
d3_gold
d3_lh
d3_kills
d3_deaths
d3_items
d4_hero
d4_level
d4_xp
d4_gold
d4_lh
d4_kills
d4_deaths
d4_items
d5_hero
d5_level
d5_xp
d5_gold
d5_lh
d5_kills
d5_deaths
d5_items
first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_tpscroll_count
radiant_boots_count
radiant_ward_observer_count
radiant_ward_sentry_count
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire

In [9]:
!mplayer ~/Музыка/tada.wav > /dev/null 

mplayer: could not connect to socket
mplayer: No such file or directory
Failed to open LIRC support. You will not be able to use your remote control.
File not found: '2'
Failed to open 2.
File not found: '1'
Failed to open 1.
